#001

In [1]:
from summarizer import Summarizer, TransformerSummarizer
summar = Summarizer()
# GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
smtokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
smmodel = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [29]:
# smtokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [3]:
# XLnet_model = TransformerSummarizer(
#     transformer_type="XLNet", transformer_model_key="xlnet-base-cased")


In [4]:
# from summarizer.sbert import SBertSummarizer
# SBert_model = SBertSummarizer('paraphrase-MiniLM-L6-v2')

In [5]:
import numpy as np
import pandas as pd
path = u"./dataset/s2t/EN_ChuanMin0225_2.srt"
# path = u"./dataset/001.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

# for i in z:
#     print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


In [6]:
import nltk
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

config_path = "./model/uncased_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/uncased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/uncased_L-12_H-768_A-12/vocab.txt'

model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)


Using TensorFlow backend.


In [7]:
pack


[['1', ['00:00:00', '00:00:03'], "You can't have a little bit of a problem"],
 ['1',
  ['00:00:03', '00:00:06'],
  'to get the same thing. And how do I agree with'],
 ['2', ['00:00:09', '00:00:09'], 'you?'],
 ['3', ['00:00:19', '00:00:23'], 'Singer?'],
 ['4',
  ['00:00:48', '00:01:04'],
  "Okay, um, before we go, ah, I'm going to discuss with you about active learning, right?"],
 ['5',
  ['00:01:04', '00:01:07'],
  "Yeah, if you've been, I presumes that, uh, you have a lady."],
 ['6',
  ['00:01:07', '00:01:15'],
  "Watch the video. And I also give you four question, but now ignore the number, because that's wrong."],
 ['7', ['00:01:15', '00:01:17'], 'So one, two, three, four. Right?'],
 ['8',
  ['00:01:17', '00:01:25'],
  'So four questions. Um, what do you understand?'],
 ['9',
  ['00:01:25', '00:01:30'],
  'But conceptual? And talk. Want to answer chopin talk.'],
 ['10',
  ['00:01:33', '00:01:36'],
  'What do you understand? The concept of chopin Talk.'],
 ['11', ['00:01:43', '00:01:

In [8]:
token = []
seg = []
token_ids = ""
segment_ids = ""
result = []

for i in range(len(pack)-1):
    if (len(pack[i][2])+len(pack[i+1][2]) >= 512):
        pack[i][2] = pack[i][2][-128:]

    token_ids, segment_ids = tokenizer.encode(
        pack[i][2], pack[i+1][2])

    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    # print(probas)
    result.append(probas)

for i in range(len(pack)-1):
    pack[i].append(result[i][0])

In [9]:
var = 0.05 #和段落數成正比

In [10]:
idxList = []
lessThanHalf = []
lessThanHalf.append('00:00:00')

for i in range(len(pack)-1):
    if(pack[i][3] < var):
        idxList.append(i)
        lessThanHalf.append(pack[i][1][1])  # 段落起始時間

In [11]:
len(lessThanHalf)

16

In [12]:
combineText = []
j = ''
j = pack[0][2]  # 段落文本
for i in range(len(pack)-1):
    # token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
    # token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
    # result = model.predict([token_testId, Seg_testId])[0][0]

    if(pack[i][3] > var):
        j += pack[i+1][2]

        if(i == len(pack)-2):
            j = pack[i+1][2]
            combineText.append(j)
            j = ""
    else:
        j += pack[i+1][2]

        combineText.append(j)
        j = ""


In [13]:
import copy
aa = ""
bb = []
for i in range(len(pack)-1):
    # print(pack[i][-1])

    if len(aa) == 0:
        aa = pack[i][2]

    if (pack[i][3] >= var):
        aa += pack[i+1][2]
    else:
        bb.append(aa)
        aa = ""

    if (i == len(pack)-2):
        # print(0)
        if len(aa) != 0:
            bb.append(aa)
        else:
            bb.append(pack[i+1][2])
combineText = bb
combineText_copy = copy.deepcopy(bb)


In [14]:
# example
# data = "All work and no play makes jack dull boy. All work and no play makes jack a dull boy."
# stopWords = set(stopwords.words('english'))
# words = word_tokenize(data)
# wordsFiltered=[]
# for w in words:
#     if w not in stopWords:
#         print(w)


In [15]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
for i in range(len(combineText)):
    combineText[i] = combineText[i].lower()

for i in range(len(combineText)):
    combineText[i] = re.sub(r"[^\w\s]", "", combineText[i])
    combineText[i] = re.sub(r"[0-9]", "", combineText[i])

# TFIDF 實作

stopWords = set(stopwords.words("english"))
words_filtered = []
for word in combineText:
    words = word_tokenize(word)
    for w in words:
        if w not in stopWords:
            words_filtered.append(w)
# ------------------------------------------
words_count = []
count = {}
for words in words_filtered:

    if words in count:
        count[words] += 1
    else:
        count[words] = 1
words_count.append(count)
# ------------------------------------------
#tf=出現在所有文檔的某個詞 / 全部文檔的詞數(包含重複)
# idf= log(所有文檔 / 某詞出現在某a個文檔) (以10為底)
# tf*idf=tfidf
keys = list(words_count[0].keys())
tf_freq = []
for i in range(len(words_count[0])):
    tf_freq.append(words_count[0][keys[i]]/len(words_filtered))
# ------------------------------------------
new_tf_freq = words_count[0].copy()
for i in range(len(keys)):
    new_tf_freq[keys[i]] = tf_freq[i]
# ------------------------------------------
idf_list = {}
for key in keys:
    idf_list[key] = 0
    for word in combineText:
        if key in word:
            idf_list[key] += 1
# ------------------------------------------
keys = list(words_count[0].keys())
inverse_document_freqency = []
for i in range(len(idf_list)):
    inverse_document_freqency.append(len(combineText)/idf_list[keys[i]])
# ------------------------------------------
keys = list(words_count[0].keys())
tfidf = np.array(tf_freq)*np.array(inverse_document_freqency)
tfidf_dict = {}
for i in range(len(keys)):
    tfidf_dict[keys[i]] = tfidf[i]
# ------------------------------------------
tfidfResult = {k: v for k, v in sorted(
    tfidf_dict.items(), key=lambda item: item[1], reverse=True)}
# ------------------------------------------
Top20TfResult = []
for i in range(20):
    Top20TfResult.append(list(tfidfResult.items())[i][0])
# ------------------------------------------
wst = []  # 分詞後文本
for sentence in combineText:
    seq_list = word_tokenize(sentence)
    wst.append(seq_list)

# ------------------------------------------

# ------------------------------------------
# ------------------------------------------
# ------------------------------------------
# ------------------------------------------


In [16]:
TF = np.zeros((np.shape(wst)[0], 20))  # 產生0陣列 np.shape(TF) = (len(wst), 20)

combineStr = "".join(combineText)

# 取詞頻前20的名詞 TFIDF

Query = Top20TfResult

for i in range(len(Query)):  # Query循環
    for j in range(len(wst)):  # wst循環
        for k in range(len(wst[j])):  # 找出wst[j]中出現了那些top20keyWord
            if(Query[i] == wst[j][k]):
                TF[j][i] = 1
                continue
# print(TF)
keyWord = []
deliver = []
for i in TF:
    for j in range(len(i)):
        if(i[j]):
            deliver.append(Query[j])
    keyWord.append(deliver)
    deliver = []

finalResult = []
tmpProcess = []
for i in range(len(combineText_copy)):
    tmpProcess.append(lessThanHalf[i])
    # tmpProcess.append(lessThanHalf[i+1])
    tmpProcess.append(combineText_copy[i])
    tmpProcess.append(keyWord[i])
    finalResult.append(tmpProcess)
    tmpProcess = []


C:\Users\Selab-001\anaconda3\envs\poem\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [17]:
for i in finalResult:
    print(i[1])

You can't have a little bit of a problemto get the same thing. And how do I agree withyou?Singer?Okay, um, before we go, ah, I'm going to discuss with you about active learning, right?Yeah, if you've been, I presumes that, uh, you have a lady.Watch the video. And I also give you four question, but now ignore the number, because that's wrong.So one, two, three, four. Right?So four questions. Um, what do you understand?But conceptual? And talk. Want to answer chopin talk.What do you understand? The concept of chopin Talk.Have a good one. Good one.David Chum Talk. Let me watch the video.No, no call. How comes you got a message that you wanted to, you know, preview that?This is the video. I-I haven't told cats.So things, you haven't watched the video, and you you don't know how to say that.UK. Have you watched the video?No, no. Why? It is our train class, the UK, you know, train class.Yeah, so, everything. So, going to post there, right again.Remember, you know, next week, you got a previe

In [18]:
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array


model = build_transformer_model(
    config_path,
    checkpoint_path
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)


In [19]:
def Euclidean_Distance(v1, v2):  # 歐式距離
    return np.sqrt(np.sum((v1-v2)**2))


In [20]:
from numpy import dot
from numpy.linalg import norm
def cosine_similarity(v1, v2):
    return dot(v1, v2)/(norm(v1)*norm(v2))
# for i in range(len(finalResult)-1):
#     print(cosine_similarity(finalResult[i][3], finalResult[i+1][3]))

In [21]:
SS = ""

for i in range(len(finalResult)):
    SS = SS.join(finalResult[i][2])  # 出現的top20keyWord
    # print(SS)
    token_ids, segment_ids = tokenizer.encode(SS)
    token_ids, segment_ids = to_array([token_ids], [segment_ids])
    finalResult[i].append(model.predict([token_ids, segment_ids])[0][0])
    #print(model.predict([token_ids, segment_ids])[0][0])
    SS = ""


In [22]:
import math
dist_result = []
for i in range(len(finalResult)-1):
    finalResult[i].append(cosine_similarity(
        finalResult[i][3], finalResult[i+1][3]))


In [23]:
# dist_result=dist_result[:-1]#沒東西可比 拿掉最後一個
for i in range(len(finalResult)-1):
    dist_result.append(finalResult[i][-1])


In [24]:
trueFinalResult = []
trueText = ""
tureTime = ''
trueKW = []
first = 0

dcvar=0.6#反比

for i in range(len(finalResult)-1):
    # print(trueFinalResult)
    if len(trueText) == 0:
        tureTime = finalResult[i][0]
        trueText = finalResult[i][1]
        trueKW = finalResult[i][2]

    if (finalResult[i][-1] <= dcvar or len(finalResult[i][2]) == 0):
        trueText += finalResult[i+1][1]
        trueKW = list(set(trueKW).union(set(finalResult[i+1][2])))
        #print(tureTime, trueText, trueKW, first)
    else:
        trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
        trueText = ""
        tureTime = ''
        trueKW = []
        #print(tureTime, trueText, trueKW, first)

    if i == len(dist_result)-1:
        if len(trueText) != 0:
            trueFinalResult.append([tureTime,
                                    trueText,
                                    trueKW
                                    ])
            #print(tureTime, trueText, trueKW, first)
        else:
            print(4)
            tureTime = finalResult[i+1][0]
            trueText = finalResult[i+1][1]
            trueKW = finalResult[i+1][2]
            trueFinalResult.append([tureTime,
                                    trueText,
                                    trueKW
                                    ])
            trueText = ""
            tureTime = ''
            trueKW = []
            #print(tureTime, trueText, trueKW, first)


# fi    rst=dist_result[i]
# se    cond=dist_result[i+1]
# if    (first<second and first*1.25>second):
#         trueText+=finalResult[i][1]+finalResult[i+1][1]
#         trueFinalResult.append(finalResult[i][0])
#         trueFinalResult.append(trueText)


4


In [25]:
for i in [trueFinalResult[0]]:
    print(i[1])
#     print('')
# len(trueFinalResult)


You can't have a little bit of a problemto get the same thing. And how do I agree withyou?Singer?Okay, um, before we go, ah, I'm going to discuss with you about active learning, right?Yeah, if you've been, I presumes that, uh, you have a lady.Watch the video. And I also give you four question, but now ignore the number, because that's wrong.So one, two, three, four. Right?So four questions. Um, what do you understand?But conceptual? And talk. Want to answer chopin talk.What do you understand? The concept of chopin Talk.Have a good one. Good one.David Chum Talk. Let me watch the video.No, no call. How comes you got a message that you wanted to, you know, preview that?This is the video. I-I haven't told cats.So things, you haven't watched the video, and you you don't know how to say that.UK. Have you watched the video?No, no. Why? It is our train class, the UK, you know, train class.Yeah, so, everything. So, going to post there, right again.Remember, you know, next week, you got a previe

In [26]:
# release gpu memory

# from numba import cuda
# cuda.select_device(0)
# cuda.close()
len(trueFinalResult)

11

In [27]:
# for i in trueFinalResult:
#     spl = i[0].split(":")
#     tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

#     spl = tmp.split(":")
#     if int(spl[0]) < 10:
#         tmp = "0"+tmp

#     full = ''.join(summar(i[1], num_sentences=10, max_length=50))
#     print(tmp, full)

# for i in trueFinalResult:
#     spl = i[0].split(":")
#     tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

#     spl = tmp.split(":")
#     if int(spl[0]) < 10:
#         tmp = "0"+tmp

#     full = ''.join(summar(i[1],ratio=0.2,min_length=10, max_length=50))
#     print(tmp, full)
for i in trueFinalResult:
    spl = i[0]
    # tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

    tmp = spl.split(":")
    if int(tmp[0]) == 0:
        spl = i[0][3:]

    full = ''.join(summar(i[1],min_length=15,max_length=50))
    # if full=='':
    #     full = ''.join(summar(i[1],min_length=10, max_length=50))
    # if full=='':
    #     full = ''.join(summar(i[1],ratio=0.2))

    print(spl, full)

00:00 Watch the video. Want to answer chopin talk. Let me watch the video. Have you watched the video?No, no. So, going to post there, right again. Um, alex, have you watched the video?No. And River said, there's no motivation.
05:19 You understand that? MR That okay, um, yes, only Teacher. And students have no motivation. And I got the the driver. Something stimulates your thinking, right? All right, the gymteacher. Oh, your problem will be different from others.
13:19 Okay, thank you.
14:13 Okay, so yeah, this, um, they summarize it. So the course, they give you the broad picture. All right, so we left cycle.
20:14 So we're going to touch thatagain. Then we know that plain analysis photo. That's right, we understand where we are. So, um, and uh, without that, become informal. In informal is not not the regular way.
26:10 And next time, you know, I don't know. I gave you the key words. But restaurants, it's great.
28:52 What is incremental? Is it a value?Everybody? All right, why shou

In [38]:
for i in trueFinalResult:
    spl = i[0]
    # tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

    tmp = spl.split(":")
    if int(tmp[0]) == 0:
        spl = i[0][3:]

    full=smtokenizer.decode(smmodel.generate(**smtokenizer(i[1], truncation=True, padding="longest", return_tensors="pt"))[0])
    # tokens = smtokenizer(text, truncation=True, padding="longest", return_tensors="pt")
    # summary = smmodel.generate(**tokens)
    # smtokenizer.decode(summary[0])
    # if full=='':
    #     full = ''.join(summar(i[1],min_length=10, max_length=50))
    # if full=='':
    #     full = ''.join(summar(i[1],ratio=0.2))

    print(spl, full)

00:00 David Chum Talk:
05:19 What can active learning teach us about motivation?
13:19 As a professor, you have to listen to your students.
14:13 We're going to talk about active learning.
20:14 We don't have enough time to describe the formal process.
26:10 The key words are sequential incrementary, irresibly incrementary, or in some other, some other way?
28:52 Let's talk about incremental.
31:00 In our series of letters from African journalists, film-maker and columnist Farai Sevenzo looks at the importance of planning.
42:18 How much do you know?
47:43 In this lesson, we're going to look at the relationship between entities.
50:46 Let's start with the relationship.


In [36]:
text = """
Python is an interpreted high-level general-purpose programming language. Its design philosophy emphasizes code readability with its use of significant indentation. Its language constructs as well as its object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.[30]

Python is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly, procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[31]

Guido van Rossum began working on Python in the late 1980s, as a successor to the ABC programming language, and first released it in 1991 as Python 0.9.0.[32] Python 2.0 was released in 2000 and introduced new features, such as list comprehensions and a garbage collection system using reference counting. Python 3.0 was released in 2008 and was a major revision of the language that is not completely backward-compatible. Python 2 was discontinued with version 2.7.18 in 2020.[33]

Python consistently ranks as one of the most popular programming languages.[34][35][36][37]"""

# tokens = smtokenizer(text, truncation=True, padding="longest", return_tensors="pt")
# summary = smmodel.generate(**tokens)
smtokenizer.decode(smmodel.generate(**smtokenizer(text, truncation=True, padding="longest", return_tensors="pt"))[0])

'Python is a programming language developed by Guido van Rossum.'

In [29]:
[trueFinalResult[0]]

[['00:00:00',
  "You can't have a little bit of a problemto get the same thing. And how do I agree withyou?Singer?Okay, um, before we go, ah, I'm going to discuss with you about active learning, right?Yeah, if you've been, I presumes that, uh, you have a lady.Watch the video. And I also give you four question, but now ignore the number, because that's wrong.So one, two, three, four. Right?So four questions. Um, what do you understand?But conceptual? And talk. Want to answer chopin talk.What do you understand? The concept of chopin Talk.Have a good one. Good one.David Chum Talk. Let me watch the video.No, no call. How comes you got a message that you wanted to, you know, preview that?This is the video. I-I haven't told cats.So things, you haven't watched the video, and you you don't know how to say that.UK. Have you watched the video?No, no. Why? It is our train class, the UK, you know, train class.Yeah, so, everything. So, going to post there, right again.Remember, you know, next week,

In [ ]:
for i in trueFinalResult:
    spl = i[0].split(":")
    tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

    spl = tmp.split(":")
    if int(spl[0]) < 10:
        tmp = "0"+tmp

    full = ''.join(SBert_model(i[1], max_length=50))
    print(tmp, full)


00:00 And David, this is our team member, tiffany. So is a use case and use case discretion.
02:25 Stay machine now and introduce our system.
05:41 Next, I will introduce our activity diagram.
14:05 And if now, I will introduce orcasta again. I'm talking about the behaviors the machine.
23:30 
25:36 This situations leads to us wasting time. After that, according to the curriculum schedule.
30:22 I have to say, it's so terrible for students. I'll talk about what here is wrong later.
39:03 Otherwise, human Detection Execution appearance. Then the recognizer request the info to database.
45:03 If you can recognize 700, recognize it, she said.
46:45 Our system is not a large and complex system.
49:52 Rapid repeatedly convert the design direction.
52:31 And look, you see you're very single now.
54:34 I will extend this part with the whole system. At the beginning, the system will start 1st.
58:13 All right, thank you for your presentation next. It will keep the cause from getting away.
61:5

In [34]:
list(set(finalResult[0][2]).union(set(finalResult[1][2])))


['course', 'um']

In [ ]:
[['00:00:00', 'course', 'um']]


In [45]:
spl = '20:40:25'.split(":")


In [46]:
spl


['20', '40', '25']

In [50]:
str(int(spl[0])*60+int(spl[1]))+":"+spl[2]


'1240:25'

In [54]:
path = "C:/Users/Selab-001/Documents/nlp/Output.txt"


In [62]:
f = open(path, "w")
f.writelines(['00:00:00', ' course,', 'um'])
f.close()


In [66]:
trueFinalResult[0][0]


'00:00:00'

In [76]:
trueFinalResult[0][2]


['course', 'um']

In [121]:
temp = []
temp.append(trueFinalResult[0][0])


In [122]:
strk = [" "+trueFinalResult[0][2][0]+",", trueFinalResult[0][2][1]]
strk


[' course,', 'um']

In [123]:
temp.extend(strk)


In [124]:
temp


['00:00:00', ' course,', 'um']

In [155]:
f = open(path, "w")
for i in range(len(trueFinalResult)):
    spl = trueFinalResult[i][0].split(":")
    tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]

    spl = tmp.split(":")
    if int(spl[0]) < 10:
        tmp = "0"+tmp
    f.writelines(tmp)
    strk = ' '
    for i in trueFinalResult[i][2]:
        strk = strk+i+','
    f.writelines(strk+"\n")
f.close()


In [139]:
spl


['00', '40', '25']

In [149]:
spl = "00:25"
spl = spl.split(":")

tmp = str(int(spl[0])*60+int(spl[1]))+":"+spl[2]
spl = tmp.split(":")


IndexError: list index out of range

In [151]:
spl = ['08', '25']


In [152]:
if int(spl[0]) < 10:
    tmp = "0"+tmp


In [153]:
tmp


'040:25'